### Algorithm Description

Implementation of the MAS approximation algorithm introduced in the paper `Maximal acyclic subgraphs and closest stable matrices`. 

In [ ]:
import numpy as np
import scipy as sc
import random
from scipy import linalg as la
from numpy.linalg import norm
import scipy.sparse as sparse
from scipy.sparse import rand as rndma
import time
import networkx as nx

In [ ]:
def lp_solution(A,v,supp,tau):
    
    D = len(supp)
    X = np.copy(A)
    
    ind = np.argsort(v)[::-1]
    ind = ind[:D]
    
    for i in xrange(dim):
        S = 0
        for l in ind:
            S += A[i,l]
            if (S <= tau):
                X[i,l] = 0
            else:
                X[i,l] = -tau + S
                break
    
    return np.round(X,prec)

In [ ]:
def forward(A,spectfin,k):
    
    while (spectfin != 0):
        
        if (spectfin > 6): 
            k += 2
        else:
            k += 1
        
        Xstar, spectfin = selective_greedy(A,k)
        
    
    return Xstar, spectfin

In [ ]:
def backward(A,X,spect,k):
    
    while (spect == 0):
        Xstar = np.copy(X)
        spectfin = spect
        k -= 1
        X, spect = selectivegreedylinf(A,k)
        
    
    return Xstar, spectfin

In [ ]:
def the_tree(A,X):
    
    
    tree = []
    vertices = [i for i in xrange(dim)]
    ind = [i for i in xrange(dim)]
    Aredux_col = np.copy(A)
    Xredux_row = np.copy(X)
    Xredux_col = np.copy(X)
    
    while (vertices != []):
    
        #finding the source(s)
        sources = [j for j in vertices if (np.sum(Xredux_row[:,j]) == 0)]
        indy = np.argmax([np.sum((Aredux_col - Xredux_col)[j]) for j in sources])
        
        the_source = sources[indy]
        
        #taking theem out
        tree.append(the_source)
        vertices.remove(the_source)
        Aredux_col = A[np.ix_(ind,vertices)]
        Xredux_col = A[np.ix_(ind,vertices)]
        Xredux_row = X[np.ix_(vertices,ind)]
       
    for x in xrange(len(tree)):
        i = tree.pop(0)
        for j in tree:
            X[i,j] = A[i,j]

    return X

In [ ]:
def ubrmthd(A):
    evals, evecs = np.linalg.eig(A)
    evals = np.real(evals)
    evals = np.round(evals,prec)
    evecs = np.round(evecs,prec)
    rho = np.amax(evals)
    rho = np.round(rho,prec)
    return evals, rho, evecs

In [ ]:
def perm(ind,D):
    
    P = np.zeros((D,D))
    
    for i in xrange(D):
        P[i,ind[i]] = 1
    
    return P

In [ ]:
def minimal(A):
    evals, rho, evecs = ubrmthd(A)
    index = np.where(evals == rho)[0]
    
    dic = {} #finding the leading eigenvectors and counting their support
    nonzeros = []
    for i in index:
        eigenvec = evecs[:,i]
        l = len(np.where(eigenvec != 0)[0])
        nonzeros.append(l)
        dic[l] = i
        
    k = dic[np.amin(nonzeros)] #fining the minimal eigenvector
    v = np.real(evecs[:,k])
    v = np.abs(v)
    v = np.round(v,prec)
    supp = list(np.where(v != 0)[0])
    return v, supp

In [ ]:
def transfer(l,m,x): #transfers element x from list l to the list m
    l.remove(x)
    m.append(x)
    
    return l, m    

In [ ]:
def isinbasic(A,index):
    
    
    i = index[0] #index that is allegedly in basic set
    void = np.where(A.T[:,i] == 0)[0] #potential void
    void = list(void)
    if i in void:
        void.remove(i)
    
    bset = [j for j in index if j not in void] #potential basic set
   
    
    
    
    bset1 = bset #set to test verices on
    while True:
        bset2 = [] #set for new vertices
        void0 = list(void)
        for k in void0:
            check = [A.T[k,j] == 0 for j in bset1]
            if not all(check):
                void, bset2 = transfer(void,bset2,k)
        if (void0 == void):
            break
        else:
            bset += bset2
            bset1 = bset2 #now testing on new vertices


    bset.sort()
    void.sort()           

    return bset, void

In [ ]:
def irreduce(A,bset):

    D = len(bset)
    C = A[np.ix_(bset,bset)]
    if (len(C) != 0):
        spectC = ubrmthd(C)[1]
    else:
        return True
    spectA = ubrmthd(A)[1]
    if (spectC == spectA):
        K = np.linalg.matrix_power(np.identity(D) + C, D-1)
        return (K > 0).all()
    else:
        return False  

In [ ]:
def isinvoid(A,index):
    
    i = index[0]
    
    
    void = np.where(A.T[i] != 0)[0] #potential void
    void = list(void)
    if i not in void:
        void.append(i)
        
    bset = [j for j in index if j not in void]
    
    void0 = list(set(void) - set([i])) #set to test verices on
    while True:
        void1 = [] #a void to be
        for k in void0:
            newvoid = [j for j in bset if A.T[k,j] != 0]
            for x in newvoid:
                bset, void = transfer(bset,void,x)
            void1 += newvoid
        if (void1 == []) or (bset == []):
            break
        else:
            void0 = void1

    bset.sort()
    void.sort()
    
    return bset, void

In [ ]:
def basicset(A,D):
    
    index = range(D) 
    
    spect = ubrmthd(A)[1] #checking for loops 
    loops = [A[i,i] for i in xrange(D)]
    frutiloop = list(np.where(spect == loops)[0])
    if (frutiloop != []):
        S = False
        bset = frutiloop
        void = list(set(index) - set(frutiloop))
    else:
        S = True
        void = []
        
    while S:
        bset, void0 = isinbasic(A,index) #potential basic and void
        if irreduce(A,bset):
            void += void0
            void = list(set(void))
            break
        else:
            bset, void0 = isinvoid(A,index)
            bset.sort()
            index = bset
            void += void0
            void = list(set(void))
            if irreduce(A,bset):
                break
                
    
                
            
    bset.sort()
    void.sort()
    ind = void + bset
    
    P = perm(ind,len(ind))
    Kac = np.matmul(P,np.matmul(A,P.T))
   
    return bset #getting the basic set

In [ ]:
def PN_greedy(A,tau):
    
    start = time.clock()
    global it

    X = np.copy(A)
    v0, supp0 = minimal(X) #computing the minimal eigenvec and its support
    it += 1
    
    while True:
        
        Z = np.copy(X)
        supp = supp0
        v = v0
        notsupp = list(set(range(dim)) - set(supp))
        notsupp.sort()
        X = lp_solution(A,v,supp,tau)
        X[notsupp] = Z[notsupp]
        
        optin = supp[:]
        
        for k in supp:
            olddot = np.dot(Z[k],v)
            newdot = np.dot(X[k],v)
            if (olddot < newdot) or (abs(olddot - newdot) < 1e-5): #see the Appendix
                X[k] = Z[k]
                optin.remove(k)
    
         
        v0, supp0 = minimal(X)
        it += 1
        if (set(supp0) == set(optin)) or (v == v0).all():
            spectra = leading(X)
            run = time.clock() - start
            print run, it
            return X, spectra
        
        else: #comparing basic set with set of optimal indices
            
            Xredux = X[np.ix_(supp,supp)]
            bsetredux = basicset(Xredux,len(supp)) #basic set
            bset = [supp[k] for k in bsetredux]
            if (set(bset) <= set(optin)):
                v0 = np.zeros(dim)
                vredux, suppredux = minimal(Xredux)
                v0[supp] = vredux #the real eigenvec
                supp0 = [supp[k] for k in suppredux]
            else:
                v0, supp0 = minimal(X)

In [ ]:
def closest_graph_PN(A):
    

    start = time.clock()
    global it
    it = 0
    

    row_sums = [np.sum(A[i]) for i in xrange(dim)]  

    k0 = np.amax(row_sums)
    print k0
    k0 = np.trunc(k0/2)
    k1 = np.amin(row_sums)
    seg = k0
    
    '''doing a bisection in k until we obtain a matrix with
    appropriate spectral radius'''

    while (seg >= 1):

        Xstar, spect_radius = PN_greedy(A,k0)
        print spect_radius, k0
        

        
        if (spect_radius > 3):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 += seg

        elif (spect_radius == 0):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 -= seg

        else:
            k1 = k0
            break

    if (spect_radius != 0):
        '''if a last obtained matrix has a spectral radius bigger than zero
        we move k forward untill we obtain an acyclic graph'''
        Xstar, spect_radius = forward(A,spect_radius,k1)
    else:
        '''if a last obtained matrix has a zero spectral radius, 
        we move k backwards untill we get a minimal k for which
        we have an acyclic graph'''
        Xstar, spect_radius = backward(A,Xstar,spect_radius,k1)


    run = time.clock() - start #running time after the Step X
    run = np.round(run,2)
    
    #computing the percentage of saved edges after the Step X
    perc = (np.sum(Xstar)/np.sum(A)) 
    
    Z = np.copy(Xstar)
    
    '''unning the DFS algorithm and restoring the edges;
    this will give us the MAS approximation'''
    Gamma = the_tree(A,Z) 


    run_tree = time.clock() - start #running time after the Step X
    run_tree = np.round(run_tree,2)
    
    '''computing the percentage of saved edges;
    this will actuall give us how good our MAS approximation is'''
    perc_tree = (np.sum(Gamma)/np.sum(A))
    
    return run_tree, perc_tree, it, norm(Gamma - A,np.Inf)

In [ ]:
def create_graph(dim):
    
    G = nx.watts_strogatz_graph(dim,25,0.1)
    G = nx.to_numpy_matrix(G)
    
    
    #making the graph directed
    
    for i in range(dim):
        for j in range(i):
            if G[i,j] == 1:
                cut = random.choice(['a','b'])
                if cut == 'a':
                    G[i,j] = 0
                elif cut == 'b':
                    G[j,i] = 0

    return G

In [ ]:
prec = 8 #setting the rounding parameter
Eps = 10**(-prec)
dim = 500 #setting the dimension
run_time = []
edge_perc = []
no_of_iter = []
for x in xrange(5):
    A = create_graph(dim)
    print leading(A)#generating start. graph
    GPN = closest_graph_PN(A) #running the algorithm 
    print GPN
    run_time.append(GPN[0])
    edge_perc.append(GPN[1])
    no_of_iter.append(GPN[2])
    print '......................'
   
    
print np.average(run_time), np.average(edge_perc), np.average(no_of_iter)